In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras import models

In [ ]:
actions = ['星期二', '幾月幾號', '台北', '星期一', '星期三', '有', '什麼', '完了嗎', '將近', '父母', '一共', '買', '家裡', '房子', '銀行', '昨天', '認識', '見她', '今天', '久', '比較', '一', '手語', '你', '他們', '星期五', '是嗎', '誰', '還沒有', '星期六', '桃園', '朋友', '高鐵到', '吃飯', '會不會呢', '我問你', '生日', '我們兩個', '租', '棒', '孩子', '零', '星期天', '明天', '運動', '星期四', '捷運站', '一樣', '上課', '我', '年齡', '無', '天氣', '相見', '名字']


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sequences = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_X_100.npy')
labels = np.load('/content/drive/MyDrive/AI Python/Dataset/Refined_New_Y_100.npy')

In [ ]:
np.array(sequences).shape 

(5500, 60, 165)

In [ ]:
np.array(labels).shape

(5500,)

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
y.shape

(5500, 55)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
y_test.shape

(275, 55)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU, Bidirectional, BatchNormalization,ReLU,Activation
from tensorflow.keras.callbacks import TensorBoard


model = Sequential()
model.add(Bidirectional(GRU(512, return_sequences=True, activation='elu'), input_shape=(60,165)))
model.add(Bidirectional(GRU(1024, return_sequences=True, activation='elu')))
model.add(Bidirectional(GRU(512, return_sequences=False, activation='elu')))
model.add(Dense(512, activation='elu'))
model.add(Dense(256, activation='elu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 60, 1024)          2085888   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 2048)          12595200  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              7870464   
_________________________________________________________________
dense (Dense)                (None, 512)               524800    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 55)                14135     
Total params: 23,221,815
Trainable params: 23,221,815
Non-trainable params: 0
____________________________________________

In [ ]:
#Bidirectional GRU (100)
from tensorflow.keras.optimizers import Adam
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
#checkpoint = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/AI Python/Dataset/Dropout.h5', monitor='val_categorical_accuracy', mode="max", save_best_only=True, verbose=1)
earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', mode="auto", patience=3, verbose=1)
rlr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, mode='auto')


model.fit(X_train, y_train,validation_split = 0.1,
              epochs=30,
              batch_size = 32,
              callbacks=[rlr,earlystopping])


Epoch 1/30
147/147 [==============================] - 112s 707ms/step - loss: 2.2843 - categorical_accuracy: 0.3805 - val_loss: 1.4668 - val_categorical_accuracy: 0.6061
Epoch 2/30
147/147 [==============================] - 103s 701ms/step - loss: 0.9713 - categorical_accuracy: 0.7099 - val_loss: 0.7054 - val_categorical_accuracy: 0.7763
Epoch 3/30
147/147 [==============================] - 106s 718ms/step - loss: 0.5492 - categorical_accuracy: 0.8311 - val_loss: 0.6685 - val_categorical_accuracy: 0.8107
Epoch 4/30
147/147 [==============================] - 105s 715ms/step - loss: 0.4773 - categorical_accuracy: 0.8530 - val_loss: 0.7574 - val_categorical_accuracy: 0.7897

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 5/30
147/147 [==============================] - 105s 716ms/step - loss: 0.1158 - categorical_accuracy: 0.9634 - val_loss: 0.2102 - val_categorical_accuracy: 0.9541
Epoch 6/30
147/147 [==============================] - 105s 715ms/ste

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'test_loss={test_loss}')
print(f'test_accuracy={test_accuracy}')

9/9 [==============================] - 1s 75ms/step - loss: 0.1510 - categorical_accuracy: 0.9745
test_loss=0.15102842450141907
test_accuracy=0.9745454788208008
